In [1]:
import asyncio
import time
import requests
from indy import anoncreds, did, ledger, pool, wallet, IndyError

import json

async def run():
    print("Supplier -> started")

    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)

    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    print("==============================")
    print("== Getting Trust Anchor credentials - Supplier getting Verinym  ==")
    print("------------------------------")

    supplier = {
        'name': 'Supplier',
        'wallet_config': json.dumps({'id': 'supplier_wallet'}),
        'wallet_credentials': json.dumps({'key': 'supplier_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }

    await create_wallet(supplier)

    (supplier['did'], supplier['key']) = await did.create_and_store_my_did(supplier['wallet'], "{}")

    response = requests.post('http://steward:5000/get_trust_anchor_verinym', data = {'entity': supplier})
    response = response.json()
    print(response)
    # supplier = response['entity']

    print("Close and Delete pool")
    await pool.close_pool_ledger(pool_['handle'])
    await pool.delete_pool_ledger_config(pool_['name'])

    print("Getting started -> done")

async def create_wallet(identity):
    print("\"{}\" -> Create wallet".format(identity['name']))
    try:
        await wallet.create_wallet(identity['wallet_config'], identity['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
            pass
    identity['wallet'] = await wallet.open_wallet(identity['wallet_config'], identity['wallet_credentials'])


if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    await run()
    time.sleep(1)  # FIXME waiting for libindy thread complete


Supplier -> started
Open Pool Ledger: pool1
== Getting Trust Anchor credentials - Supplier getting Verinym  ==
------------------------------
"Supplier" -> Create wallet


TypeError: 'Response' object is not subscriptable